In [1]:
import numpy as np
import pandas as pd

np.random.seed(42)

n = 7*24*60  # 7 jours en minutes
dates = pd.date_range(start="2023-01-01", periods=n, freq="T")

btc_log_returns = np.random.normal(0, 0.0005, n)
btc_prices = 20000 * np.exp(np.cumsum(btc_log_returns))

noise = np.random.normal(0, 0.0007, n)
eth_log_returns = 0.6 * btc_log_returns + noise * 0.4
eth_prices = 1500 * np.exp(np.cumsum(eth_log_returns))

df = pd.DataFrame({"BTC": btc_prices, "ETH": eth_prices}, index=dates)

initial_capital = 10000
cash = initial_capital
position = 0
entry_price = 0
equity_curve = []

lag_minutes = 10
window = 120
min_beta_threshold = 0.005  # seuil plus bas = plus de trades
stop_loss_pct = 0.005       # stop loss à 0.5%
take_profit_pct = 0.07      # take profit à 7%
risk_allocation = 0.4       # 40% du capital à chaque trade (plus agressif)
leverage = 2                # levier 2x

df['ETH_log'] = np.log(df['ETH'])
df['ETH_diff'] = df['ETH_log'].diff().abs()
df['ATR'] = df['ETH_diff'].rolling(window=14).mean()

btc_log = np.log(df["BTC"]).diff()
eth_log = np.log(df["ETH"]).diff()

last_entry_equity = None
exit_time = None

for i in range(window, len(df) - lag_minutes):
    btc_window = btc_log.iloc[i - window + 1 : i + 1].dropna()
    eth_window = eth_log.iloc[i - window + 1 - lag_minutes : i + 1 - lag_minutes].dropna()
    
    if len(btc_window) == len(eth_window) and len(btc_window) > 1:
        beta = np.cov(btc_window, eth_window)[0, 1] / np.var(btc_window)
    else:
        beta = 0

    atr = df['ATR'].iloc[i]
    if np.isnan(atr) or atr == 0:
        atr = 0.0005

    if beta > min_beta_threshold and position == 0:
        risk_per_trade = risk_allocation * cash
        pos_size = risk_per_trade / (atr * df["ETH"].iloc[i])
        pos_size = min(pos_size, cash / df["ETH"].iloc[i])  

        if pos_size > 0:
            pos_size *= leverage  # appliquer levier
            entry_price = df["ETH"].iloc[i]
            position = pos_size
            cash -= pos_size / leverage * entry_price  # mise cash réduite car levier
            last_entry_equity = cash + position * entry_price
            print(f"Entry {df.index[i]} beta={beta:.4f} size={pos_size:.4f} price={entry_price:.2f}")

    if position != 0:
        current_price = df["ETH"].iloc[i]
        pnl_pct = (current_price - entry_price) / entry_price

        if pnl_pct <= -stop_loss_pct or pnl_pct >= take_profit_pct:
            cash += position / leverage * current_price
            print(f"Exit {df.index[i]} pnl={pnl_pct*100:.2f}% cash={cash:.2f}")
            position = 0
            entry_price = 0

    equity = cash + (position * df["ETH"].iloc[i] if position != 0 else 0)
    equity_curve.append(equity)

equity_curve = pd.Series(equity_curve, index=df.index[window : window + len(equity_curve)])
returns = equity_curve.pct_change().dropna()
total_return = (equity_curve.iloc[-1] / initial_capital - 1) * 100
sharpe_ratio = returns.mean() / returns.std() * np.sqrt(60*24*30)

print(f"Total Return sur 7 jours = {total_return:.2f} %")
print(f"Sharpe Ratio estimé (annualisé) = {sharpe_ratio:.2f}")


C:\Users\redwan.khafif\AppData\Local\Temp\ipykernel_12868\2948216024.py:7: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  dates = pd.date_range(start="2023-01-01", periods=n, freq="T")


Entry 2023-01-01 02:19:00 beta=0.0147 size=13.4118 price=1491.23
Exit 2023-01-01 11:16:00 pnl=-0.52% cash=9948.15
Entry 2023-01-01 11:17:00 beta=0.0524 size=13.4052 price=1484.22
Total Return sur 7 jours = 107.26 %
Sharpe Ratio estimé (annualisé) = 2.13
